# General Imports

    !! IMPORTANT !!
    If you did NOT install opengrid with pip, 
    make sure the path to the opengrid folder is added to your PYTHONPATH

In [ ]:
import os
import inspect
import sys
import pandas as pd
import charts

from opengrid.library import houseprint

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

## Houseprint

In [ ]:
hp = houseprint.Houseprint()
# for testing:
# hp = houseprint.Houseprint(spreadsheet='unit and integration test houseprint')

In [ ]:
hp

In [ ]:
hp.sites[:5]

In [ ]:
hp.get_devices()[:4]

In [ ]:
hp.get_sensors('water')[:3]

A Houseprint object can be saved as a pickle. It loses its tmpo session however (connections cannot be pickled)

In [ ]:
hp.save('new_houseprint.pkl')

In [ ]:
hp = houseprint.load_houseprint_from_file('new_houseprint.pkl')

### TMPO

The houseprint, sites, devices and sensors all have a get_data method. In order to get these working for the fluksosensors, the houseprint creates a tmpo session.

In [ ]:
hp.init_tmpo()
hp._tmpos.debug = False
hp.sync_tmpos()

## Lookup sites, devices, sensors based on key

These methods return a single object

In [ ]:
hp.find_site(1)

In [ ]:
hp.find_device('FL03001562')

In [ ]:
sensor = hp.find_sensor('d5a747b86224834f745f4c9775d70241')

In [ ]:
print(sensor.site)
print(sensor.unit)

## Lookup sites, devices, sensors based on search criteria

These methods return a list with objects satisfying the criteria

In [ ]:
hp.search_sites(inhabitants=5)

In [ ]:
hp.search_sensors(type='electricity', direction='Import')

### Get Data

In [ ]:
hp.sync_tmpos()

In [ ]:
head = pd.Timestamp('20151102')
tail = pd.Timestamp('20151103')
df = hp.get_data(sensortype='water', head=head, tail=tail, diff=True, resample='min', unit='l/min')
charts.plot(df, stock=True, show='inline')

## Site

In [ ]:
site = hp.find_site(1)
site

In [ ]:
print(site.size)
print(site.inhabitants)
print(site.postcode)
print(site.construction_year)
print(site.k_level)
print(site.e_level)
print(site.epc_cert)

In [ ]:
site.devices

In [ ]:
site.get_sensors('electricity')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150628')
df=site.get_data(sensortype='electricity', head=head,tail=tail, diff=True, unit='kW')
charts.plot(df, stock=True, show='inline')

## Device

In [ ]:
device = hp.find_device('FL03001552')
device

In [ ]:
device.key

In [ ]:
device.get_sensors('gas')

In [ ]:
head = pd.Timestamp('20151101')
tail = pd.Timestamp('20151104')
df = hp.get_data(sensortype='gas', head=head,tail=tail, diff=True, unit='kW')
charts.plot(df, stock=True, show='inline')

## Sensor

In [ ]:
sensor = hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')
sensor

In [ ]:
sensor.key

In [ ]:
sensor.type

In [ ]:
sensor.description

In [ ]:
sensor.system

In [ ]:
sensor.unit

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
df=sensor.get_data(head,tail,diff=True, unit='W')
charts.plot(df, stock=True, show='inline')

## Getting data for a selection of sensors

In [ ]:
sensors = hp.search_sensors(type='electricity', system='solar')
print(sensors)
df = hp.get_data(sensors=sensors, head=head, tail=tail, diff=True, unit='W')
charts.plot(df, stock=True, show='inline')

## Dynamically loading data sensor per sensor

A call to `hp.get_data()` is lazy: it creates a big list of Data Series per sensor and concatenates them. This can take a while, specifically when you need many sensors and a large time span.

Often, you don't use the big DataFrame as a whole, you rather re-divide it by using a for loop and looking at each sensor individually.

By using `hp.get_data_dynamic()`, data is fetched from tmpo per sensor at a time, just when you need it.

In [ ]:
dyn_data = hp.get_data_dynamic(sensortype='electricity', head=head, tail=tail)

In [ ]:
ts = next(dyn_data)
df = pd.DataFrame(ts)
charts.plot(df, stock=True, show='inline')

You can run the cell above multiple times and eacht time the next sensor will be fetched